In [1]:
import numpy as np
import pandas as pd

In [7]:
market_data = pd.read_excel("../../raw_data/market_data.xlsx")
market_data.drop(index=market_data.index[0], axis=0, inplace=True)
market_data = market_data.reset_index(drop = True)

In [8]:
market_data.head()

,Time (UTC+10),Regions NSW Trading Price ($/MWh),Regions SA Trading Price ($/MWh),Regions TAS Trading Price ($/MWh),Regions VIC Trading Price ($/MWh),Regions NSW Trading Total Intermittent Generation (MW),Regions SA Trading Total Intermittent Generation (MW),Regions TAS Trading Total Intermittent Generation (MW),Regions VIC Trading Total Intermittent Generation (MW),Regions NSW Operational Demand (MW),Regions SA Operational Demand (MW),Regions TAS Operational Demand (MW),Regions VIC Operational Demand (MW)
0,2018-01-01 00:30:00,91.86,107.17,92.28,92.46,0.15,43.07,118.73,131.68,6974,1359,1082,4398
1,2018-01-01 01:00:00,88.83,103.31,87.53,87.62,0.13,41.67,110.48,119.98,6790,1316,1071,4238
2,2018-01-01 01:30:00,73.62,88.20,76.29,73.08,0.14,42.15,120.09,123.86,6536,1240,1067,4112
3,2018-01-01 02:00:00,71.49,85.24,75.10,70.18,0.16,38.31,114.64,132.72,6339,1194,1061,3956
4,2018-01-01 02:30:00,69.27,81.75,72.92,67.43,0.16,33.39,112.90,120.73,6195,1163,1060,3833


### Adding Period

In [9]:
# Create a period for a whole day which are 48 as
# Spot prices are taken by the 30 minutes mark.
period = []
count = 1
for i in range(1, len(market_data) + 1):
    period.append(count)
    count += 1
    if (i % 48) == 0:
        count = 1
        
market_data['Period'] = pd.Series(period)
market_data['Time (UTC+10)'] = pd.to_datetime(market_data['Time (UTC+10)'])

In [29]:
# TRAINING SET
# Training period is from 01/01/2018 to 30/06/2021 provided in the spec
train_start_period = '2021-01-07 00:00:00'
train_end_period   = '2021-01-10 00:00:00'

train_period = (market_data['Time (UTC+10)'] >= train_start_period) & \
               (market_data['Time (UTC+10)'] <= train_end_period)

In [30]:
train_data = market_data[train_period]
train_data.head(48)

,Time (UTC+10),Regions NSW Trading Price ($/MWh),Regions SA Trading Price ($/MWh),Regions TAS Trading Price ($/MWh),Regions VIC Trading Price ($/MWh),Regions NSW Trading Total Intermittent Generation (MW),Regions SA Trading Total Intermittent Generation (MW),Regions TAS Trading Total Intermittent Generation (MW),Regions VIC Trading Total Intermittent Generation (MW),Regions NSW Operational Demand (MW),Regions SA Operational Demand (MW),Regions TAS Operational Demand (MW),Regions VIC Operational Demand (MW),Period
52895,2021-01-07 00:00:00,36.26,30.33,36.23,31.13,69.96,186.62,42.04,269.49,7134,1506,1031,4607,48
52896,2021-01-07 00:30:00,38.20,14.27,35.34,14.12,52.07,179.18,39.31,270.67,6967,1513,1029,4466,1
52897,2021-01-07 01:00:00,37.87,-14.26,35.34,-13.94,38.64,189.33,35.67,268.66,6765,1503,1025,4328,2
52898,2021-01-07 01:30:00,35.09,-16.24,35.32,-16.05,37.38,189.18,35.99,261.06,6479,1417,1017,4241,3
52899,2021-01-07 02:00:00,35.35,-24.17,35.34,-24.31,45.20,182.02,30.75,252.77,6237,1352,1040,4100,4
52900,2021-01-07 02:30:00,35.47,-20.66,35.34,-20.80,45.77,170.17,26.01,249.32,6094,1325,1043,4021,5
52901,2021-01-07 03:00:00,35.07,0.02,35.34,0.02,35.19,150.43,26.37,273.06,6030,1309,1035,3968,6
52902,2021-01-07 03:30:00,35.04,-8.43,35.34,-8.81,38.67,153.76,26.95,231.08,6010,1280,1038,3919,7
52903,2021-01-07 04:00:00,35.47,-29.16,37.07,-30.46,35.30,170.31,29.12,223.48,6052,1260,1042,3933,8
52904,2021-01-07 04:30:00,33.11,-7.78,38.04,-7.85,34.59,147.75,28.55,209.99,6135,1253,1054,4022,9
